In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE

# 1. 加载数据集
df = pd.read_csv('normalized_dataset.csv')

# 2. 分离特征和目标变量
# 假设 'FailureMode' 为目标变量；'Unnamed: 0' 和 'Section' 不参与训练
X = df.drop(['Unnamed: 0', 'FailureMode', 'Section'], axis=1)
y = df['FailureMode']

# 3. 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. 对训练集应用 SMOTE 进行过采样（平衡类别）
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# 5. 使用 PCA 降维
# 保留 95% 的方差，可以根据需要调整 n_components
pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_smote)
X_test_pca = pca.transform(X_test)

# 6. 定义 SVM 模型的参数网格
param_grid = {
    'C': [20, 15, 10, 100],         # 惩罚参数
    'gamma': [1, 1.1, 1.01, 1.001],   # 核函数参数
    'kernel': ['rbf']               # 使用径向基核函数
}

# 7. 使用 GridSearchCV 结合 5 折交叉验证训练 SVM 模型
grid = GridSearchCV(SVC(), param_grid, cv=10, verbose=2, n_jobs=-1)
grid.fit(X_train_pca, y_train_smote)

# 输出最佳参数和模型
print("最佳参数配置:", grid.best_params_)
print("最佳模型:", grid.best_estimator_)

# 8. 在测试集上预测，并输出评估指标
y_pred = grid.predict(X_test_pca)

print("\n混淆矩阵:")
print(confusion_matrix(y_test, y_pred))

print("\n分类报告:")
print(classification_report(y_test, y_pred))

print("\n准确率:", accuracy_score(y_test, y_pred))


Fitting 10 folds for each of 16 candidates, totalling 160 fits
最佳参数配置: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
最佳模型: SVC(C=10, gamma=1)

混淆矩阵:
[[33  3  1  0]
 [ 2  5  2  0]
 [ 0  1 33  0]
 [ 0  1  1  4]]

分类报告:
              precision    recall  f1-score   support

           1       0.94      0.89      0.92        37
           2       0.50      0.56      0.53         9
           3       0.89      0.97      0.93        34
           4       1.00      0.67      0.80         6

    accuracy                           0.87        86
   macro avg       0.83      0.77      0.79        86
weighted avg       0.88      0.87      0.87        86


准确率: 0.872093023255814
